In [56]:
import numpy as np
import pandas as pd
import plotly.express as px
import kaggle
import os
import tensorflow as tf

# Model Plan:
- Find a api for real time stock data
- Find a api for real time news
- Feature engineer features
- Build Multi-modal model for day trading.
- First branch of model will be Time Series analysis model that will take the real time stock data as a sequence
- Second branch of model will be a text classifier/sentiment analysis model that can take the real time news
- Combine the two branches to make the multi-modal model

# Agent Plan:
- Use same apis to get input data (This is how agent recieves input)
- Use LLM with function calling to helper function and to run the model (Thinking part)
- Have the agent output the result or make the trade itself using the api (The action/goal achieving part)


## Data Collection

In [26]:
list_of_tickers = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 'META', 'GOOGL', 'TSLA']

In [55]:
datasets = [
    "varpit94/apple-stock-data-updated-till-22jun2021",
    "prajwaldongre/nvidia-corp-share-price-2000-2024",
    "varpit94/microsoft-stock-data",
    "guillemservera/amzn-stock-data",
    "henryshan/meta-platforms-inc-meta",
    "henryshan/google-stock-price",
    "guillemservera/tsla-stock-data"
]
for dataset in datasets: 
    kaggle.api.dataset_download_files(dataset, path='/home/misha/Desktop/data/day_trader/', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/varpit94/apple-stock-data-updated-till-22jun2021
Dataset URL: https://www.kaggle.com/datasets/prajwaldongre/nvidia-corp-share-price-2000-2024
Dataset URL: https://www.kaggle.com/datasets/varpit94/microsoft-stock-data
Dataset URL: https://www.kaggle.com/datasets/guillemservera/amzn-stock-data
Dataset URL: https://www.kaggle.com/datasets/henryshan/meta-platforms-inc-meta
Dataset URL: https://www.kaggle.com/datasets/henryshan/google-stock-price
Dataset URL: https://www.kaggle.com/datasets/guillemservera/tsla-stock-data


In [73]:
df = pd.DataFrame()
for file in os.listdir('/home/misha/Desktop/data/day_trader/'):
    data = pd.read_csv('/home/misha/Desktop/data/day_trader/' + file)
    data['filename'] = file
    df = pd.concat([df, data], axis=0, ignore_index=True)

In [74]:
df.drop(['raw_close', 'change_percent', 'avg_vol_20d', 'adjusted_close', 'Adj Close'], axis=1, inplace=True)

In [76]:
df_cleaned = df.drop(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume.1'], errors='ignore')

# Optional: Rename columns to capitalize if needed
df_cleaned.rename(columns={
    'date': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'
}, inplace=True)


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53862 entries, 0 to 53861
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      20490 non-null  object 
 1   open      20490 non-null  float64
 2   high      20490 non-null  float64
 3   low       20490 non-null  float64
 4   close     20490 non-null  float64
 5   volume    20490 non-null  float64
 6   filename  53862 non-null  object 
 7   Date      33372 non-null  object 
 8   Open      33372 non-null  float64
 9   High      33372 non-null  float64
 10  Low       33372 non-null  float64
 11  Close     33372 non-null  float64
 12  Volume    33372 non-null  float64
dtypes: float64(10), object(3)
memory usage: 5.3+ MB
